In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(font_scale=1.35)

from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.metrics import mean_squared_error, make_scorer, accuracy_score
from sklearn.model_selection import cross_val_score, GridSearchCV, cross_val_predict
from sklearn.model_selection import KFold, StratifiedKFold, RepeatedKFold, RepeatedStratifiedKFold
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from lightgbm import LGBMClassifier

In [2]:
data_train = pd.read_csv('train_classification.csv')

In [3]:
# Response rate
data_train.host_response_rate = data_train.host_response_rate.str.replace('%','').astype(float)

# Host acceptance rate
data_train.host_acceptance_rate = data_train.host_acceptance_rate.str.replace('%','').astype(float)

In [4]:
# Imputing numeric

data_train = data_train.fillna(data_train.median())

C:\Users\julia\AppData\Local\Temp\ipykernel_10808\2115613911.py:3: FutureWarning: The default value of numeric_only in DataFrame.median is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  data_train = data_train.fillna(data_train.median())


In [5]:
# apply everythingnto the test data
data_test = pd.read_csv('test_classification.csv')
data_test.host_response_rate = data_test.host_response_rate.str.replace('%','').astype(float)
data_test.host_acceptance_rate = data_test.host_acceptance_rate.str.replace('%','').astype(float)
data_test = data_test.fillna(data_test.median())

C:\Users\julia\AppData\Local\Temp\ipykernel_10808\3534192112.py:5: FutureWarning: The default value of numeric_only in DataFrame.median is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  data_test = data_test.fillna(data_test.median())


In [6]:
data_train['host_is_superhost'] = data_train['host_is_superhost'].replace({'t': 1, 'f': 0})

data_train['host_response_time'] = data_train['host_response_time'].apply(lambda time: 1 if time == 'within an hour' or time == 'within a few hours' else 0)
data_test['host_response_time'] = data_test['host_response_time'].apply(lambda time: 1 if time == 'within an hour' or time == 'within a few hours' else 0)

data_train['room_type'] = data_train['room_type'].apply(lambda room: 1 if room in ['Private room', 'Entire home/apt', 'Hotel room'] else 0)
data_test['room_type'] = data_test['room_type'].apply(lambda room: 1 if room in ['Private room', 'Entire home/apt', 'Hotel room'] else 0)

data_train['property_type'] = data_train['property_type'].apply(lambda location: 1 if location in ['Shared room in home', 'Shared room in bungalow', 'Shared room in hostel', 'Shared room in rental unit', 'Shared room in condo'] else 0)
data_test['property_type'] = data_test['property_type'].apply(lambda location: 1 if location in ['Shared room in home', 'Shared room in bungalow', 'Shared room in hostel', 'Shared room in rental unit', 'Shared room in condo'] else 0)

data_train['host_neighbourhood'] = data_train['host_neighbourhood'].apply(lambda neighborhood: 1 if neighborhood in ['Lakeview', 'Mount Greenwood', 'Gold Coast', 'Chicago Loop', 'Edison Park'] else 0)
data_test['host_neighbourhood'] = data_test['host_neighbourhood'].apply(lambda neighborhood: 1 if neighborhood in ['Lakeview', 'Mount Greenwood', 'Gold Coast', 'Chicago Loop', 'Edison Park'] else 0)

data_train['host_total_listings_count'] = data_train['host_total_listings_count'].apply(lambda count: 1 if count <= 7 else 0)
data_test['host_total_listings_count'] = data_test['host_total_listings_count'].apply(lambda count: 1 if count <= 7 else 0)

data_train['bathrooms_text'] = data_train['bathrooms_text'].str.extract(r'(\d+)').astype(float)
data_test['bathrooms_text'] = data_test['bathrooms_text'].str.extract(r'(\d+)').astype(float)
data_train['bathrooms_text'] = data_train['bathrooms_text'].fillna(data_train['bathrooms_text'].median())
data_test['bathrooms_text'] = data_test['bathrooms_text'].fillna(data_test['bathrooms_text'].median())


In [7]:
# Convert float values to string
data_train['host_location'] = data_train['host_location'].astype(str)
data_test['host_location'] = data_test['host_location'].astype(str)

# List of cities to be considered as 'West'
west_cities = ['Chicago, IL', 'Lakeview', 'Mount Greenwood', 'Gold Coast', 'Chicago Loop', 'Edison Park']

# Map locations to 'West' (1), 'East' (0), and NaN (0)
data_train['host_location'] = data_train['host_location'].apply(lambda location: 1 if any(city in location for city in west_cities) else (0 if location == 'nan' else 0))
data_test['host_location'] = data_test['host_location'].apply(lambda location: 1 if any(city in location for city in west_cities) else (0 if location == 'nan' else 0))

data_train['host_has_profile_pic'] = data_train['host_has_profile_pic'].map({'t': 1, 'f': 0})
data_test['host_has_profile_pic'] = data_test['host_has_profile_pic'].map({'t': 1, 'f': 0})

data_train['host_identity_verified'] = data_train['host_identity_verified'].map({'t': 1, 'f': 0})
data_test['host_identity_verified'] = data_test['host_identity_verified'].map({'t': 1, 'f': 0})

data_train['has_availability'] = data_train['has_availability'].map({'t': 1, 'f': 0})
data_test['has_availability'] = data_test['has_availability'].map({'t': 1, 'f': 0})

data_train['instant_bookable'] = data_train['instant_bookable'].map({'t': 1, 'f': 0})
data_test['instant_bookable'] = data_test['instant_bookable'].map({'t': 1, 'f': 0})

data_train['host_verifications'] = data_train['host_verifications'].apply(lambda verifications: 1 if 'email' in verifications or 'phone' in verifications else 0)
data_test['host_verifications'] = data_test['host_verifications'].apply(lambda verifications: 1 if 'email' in verifications or 'phone' in verifications else 0)

neighborhoods_to_map_to_1 = ['Lake View', 'Lincoln Park', 'Near North Side', 'West Town', 'Logan Square']  # Add more neighborhoods as needed

data_train['neighbourhood_cleansed'] = data_train['neighbourhood_cleansed'].apply(lambda neighborhood: 1 if neighborhood in neighborhoods_to_map_to_1 else 0)
data_test['neighbourhood_cleansed'] = data_test['neighbourhood_cleansed'].apply(lambda neighborhood: 1 if neighborhood in neighborhoods_to_map_to_1 else 0)

columns_to_exclude = ['host_id', 'host_since', 'first_review', 'last_review']

data_train = data_train.drop(columns=columns_to_exclude, errors='ignore')
data_test = data_test.drop(columns=columns_to_exclude, errors='ignore')


In [8]:

data_train = data_train.fillna(data_train.median())
data_test = data_test.fillna(data_test.median())

In [9]:
X_train = data_train.drop(columns='host_is_superhost')
y_train = data_train['host_is_superhost']

X_test = data_test

In [10]:
model = LGBMClassifier(random_state = 12, verbose = -1)

grid = {
            'n_estimators':[100,500,1000],
            'max_depth':[4,6,8],
            'learning_rate':[0.001,0.01,0.1], # Different orders of magnitude
            'subsample': [0.5,0.75,1.0], # Different floats between 0.5 and 1
            'reg_lambda':[0.01,0.1,1], # Try different orders of magnitude - maybe 0.001
            'colsample_bytree':[0.5,0.75,1]
}

gscv = GridSearchCV(model, grid, cv=3, scoring = 'neg_root_mean_squared_error', verbose = 0)

gscv.fit(X_train, y_train)

GridSearchCV(cv=3, estimator=LGBMClassifier(random_state=12, verbose=-1),
             param_grid={'colsample_bytree': [0.5, 0.75, 1],
                         'learning_rate': [0.001, 0.01, 0.1],
                         'max_depth': [4, 6, 8],
                         'n_estimators': [100, 500, 1000],
                         'reg_lambda': [0.01, 0.1, 1],
                         'subsample': [0.5, 0.75, 1.0]},
             scoring='neg_root_mean_squared_error')

In [11]:
gscv.best_params_

{'colsample_bytree': 0.75,
 'learning_rate': 0.1,
 'max_depth': 8,
 'n_estimators': 500,
 'reg_lambda': 0.1,
 'subsample': 0.5}

In [12]:
feature_importances = gscv.best_estimator_.feature_importances_


selected_features = X_train.columns[np.where(feature_importances > 0)]

X_train_selected = X_train[selected_features]

len(selected_features)

46

In [13]:
grid = {
            'n_estimators':[400,500,600],
            'max_depth':[7,8,9],
            'learning_rate':[0.01,0.1,1], # Different orders of magnitude
            'subsample': [0.5,0.55,0.6], # Different floats between 0.5 and 1
            'reg_lambda':[0.01,0.1,1], # Try different orders of magnitude - maybe 0.001
            'colsample_bytree':[0.6,0.75,0.8]
}

gscv = GridSearchCV(model, grid, cv=3, scoring = 'neg_root_mean_squared_error', verbose = 0)

gscv.fit(X_train_selected, y_train)

GridSearchCV(cv=3, estimator=LGBMClassifier(random_state=12, verbose=-1),
             param_grid={'colsample_bytree': [0.6, 0.75, 0.8],
                         'learning_rate': [0.01, 0.1, 1],
                         'max_depth': [7, 8, 9],
                         'n_estimators': [400, 500, 600],
                         'reg_lambda': [0.01, 0.1, 1],
                         'subsample': [0.5, 0.55, 0.6]},
             scoring='neg_root_mean_squared_error')

In [14]:
gscv.best_params_

{'colsample_bytree': 0.75,
 'learning_rate': 0.1,
 'max_depth': 9,
 'n_estimators': 600,
 'reg_lambda': 0.1,
 'subsample': 0.5}

In [15]:
# Threshold tuning

y_pred_probs = cross_val_predict(gscv.best_estimator_, X_train_selected, y_train, cv = 5, method = 'predict_proba')[:,1]

thrs = np.arange(0,1.01,0.001)

accs = []

for thr in thrs:
      accs.append(accuracy_score(y_train, y_pred_probs > thr))

In [16]:
best_thr = thrs[accs.index(max(accs))]

best_thr

0.457

In [17]:
gscv.best_params_

{'colsample_bytree': 0.75,
 'learning_rate': 0.1,
 'max_depth': 9,
 'n_estimators': 600,
 'reg_lambda': 0.1,
 'subsample': 0.5}

In [19]:
model = LGBMClassifier(random_state = 12, n_estimators=600,learning_rate=0.1,max_depth=11,reg_lambda=0.1,subsample=0.5, verbose = -1)

model.fit(X_train_selected, y_train)

LGBMClassifier(max_depth=11, n_estimators=600, random_state=12, reg_lambda=0.1,
               subsample=0.5, verbose=-1)

In [20]:
X_test_selected = X_test[selected_features]

# Make predictions on the test data
preds = model.predict_proba(X_test_selected)[:,1] > best_thr

# Create a DataFrame with predictions and save it to a CSV file
output = pd.DataFrame({'id': data_test.id, 'predicted': preds})
output.to_csv('LightGBM_classification_submission.csv', index=False)